In [1]:
# ML library
from datetime import datetime
start_time = datetime.now()

import matplotlib.cbook as cbook

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display as d
from IPython.display import Audio

import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import sys
from nltk.corpus import stopwords
import re

print('LOAD DURATION: ', datetime.now() - start_time) # about a minute

LOAD DURATION:  0:00:02.812432


In [ ]:
data=pd.read_csv('DatingAppReviewsDataset.csv')

# Preprocessing

In [34]:
data=data.iloc[:,1:]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681994 entries, 0 to 681993
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Name       681989 non-null  object
 1   Review     680642 non-null  object
 2   Rating     681994 non-null  int64 
 3   #ThumbsUp  681994 non-null  int64 
 4   Date&Time  681994 non-null  object
 5   App        681994 non-null  object
dtypes: int64(2), object(4)
memory usage: 31.2+ MB


In [35]:
# drop duplicate and empty review
data.drop_duplicates('Review', inplace=True)
data=data.dropna(subset=['Review'])
data=data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518394 entries, 0 to 518393
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Name       518389 non-null  object
 1   Review     518394 non-null  object
 2   Rating     518394 non-null  int64 
 3   #ThumbsUp  518394 non-null  int64 
 4   Date&Time  518394 non-null  object
 5   App        518394 non-null  object
dtypes: int64(2), object(4)
memory usage: 23.7+ MB


In [36]:
# Remove "\n", "<br />"

data['Review']=data['Review'].str.replace('\n','')
data['Review']=data['Review'].str.replace("<br />",' ')
data['Review']=data['Review'].str.replace(r"\s\s+",' ') # replace duplicate space into single space
# lines[i]=re.sub("[0-9]","",lines[i])

C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_52072/3733981557.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Review']=data['Review'].str.replace(r"\s\s+",' ') # replace duplicate space into single space


## Deal With emoji,urls and so on

In [38]:
from textacy.preprocessing.replace import urls, hashtags, numbers, emails, emojis, currency_symbols # change these kinds of words
data["Review"] = data["Review"].apply(urls).apply(hashtags).apply(currency_symbols).apply(emojis).apply(emails)

## Detect Different Language

In [40]:
%%time
# language detection
# time: 52 mins for all review
from langdetect import detect
from langdetect import detect_langs
lang=[]
for i in range(len(data)):

    try:
        lang.append(detect(data.loc[i,'Review']))
    except:
        lang.append('undetect')



Wall time: 52min 55s


In [41]:
data['language']=lang

In [ ]:
# lowercase text
data['Review']=data['Review'].str.lower()

## Get English Review

In [44]:
data.to_csv('review_with_language.csv',index=None)

In [3]:
data=pd.read_csv('review_with_language.csv')

In [4]:
data_en=data.loc[data['language']=='en',:]
data_en.reset_index(drop=True,inplace=True)
data_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461947 entries, 0 to 461946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Name       461942 non-null  object
 1   Review     461947 non-null  object
 2   Rating     461947 non-null  int64 
 3   #ThumbsUp  461947 non-null  int64 
 4   Date&Time  461947 non-null  object
 5   App        461947 non-null  object
 6   language   461947 non-null  object
dtypes: int64(2), object(5)
memory usage: 24.7+ MB


## Lemmatization and Remove Stop Words

In [6]:

# change some modal verb because when doing lemmatization, it will change "shouldn't" to "should n't"
data_en['Review']=data_en['Review'].str.replace(r"(can)'t",r'\1 not')
data_en['Review']=data_en['Review'].str.replace(r"(should|has|have|did|do|does|is|are|was|were|could|must)n't",r'\1 not')
data_en['Review']=data_en['Review'].str.replace(r"(should|has|have|did|do|does|is|are|was|were|could|must|can) not",r'\1 not')

# i'm -> i am| it's -> it is 
data_en['Review']=data_en['Review'].str.replace(r"(i)'m",r'\1 am')
data_en['Review']=data_en['Review'].str.replace(r"(i)'ll",r'\1 will')
data_en['Review']=data_en['Review'].str.replace(r"(it)'s",r'\1 is')
data_en['Review']=data_en['Review'].str.replace(r"(you)'re",r'\1 are')
data_en['Review']=data_en['Review'].str.replace(r"(i|you|they)'ve",r'\1 have')


C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_3216/2717216701.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_en['Review']=data_en['Review'].str.replace(r"(can)'t",r'\1 not')
C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_3216/2717216701.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en['Review']=data_en['Review'].str.replace(r"(can)'t",r'\1 not')
C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_3216/2717216701.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_en['Review']=data_en['Review'].str.replace(r"(should|has|have|did|do|does|is|are|was|were|could|must)n't",r'\1 not')
C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_32

In [7]:
## lemmatization
## https://gist.github.com/gaurav5430/9fce93759eb2f6b1697883c3782f30de#file-nltk-lemmatize-sentences-py
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [8]:
# lemmatize
# time:17mins
# time : 206mins
for i in range(len(data_en)):
    data_en.loc[i,'Review']=lemmatize_sentence(data_en.loc[i,'Review'])

d:\anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
data_en.to_csv('reivew_after_lemma.csv',index=None)

## Use Regex to Group Word

In [57]:

# group similar meanings words

data_en['Review']=data_en['Review'].str.replace(r"\b(?:cancel|cancelled|canceled)\b",'cancel',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:location|position)\b",'location',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:block|ban)\b",'block',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:respond|reply|reponse)\b",'response',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:love|like|perfer)\b",'love',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:wrong|error|mistake)\b",'error',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:bot|robot)\b",'robot',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:issue|problem)\b",'issue',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:cheat|fraud|scam|fake)\b",'scam',case=False) 

data_en['Review']=data_en['Review'].str.replace(r"\b(?:men|man|male|boy)\b",'male',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:women|woman|female|girl|lady)\b",'female',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:date|dating)\b",'dating',case=False) 
data_en['Review']=data_en['Review'].str.replace(r"\b(?:real|true)\b",'real',case=False) 


data_en['Review']=data_en['Review'].str.replace(r'\b(?:years?-? ?|yrs?-? ?)(?:old)\b','year old',case=False)


data_en['Review']=data_en['Review'].str.replace(r'\b(?:excellent|good|great|well|perfect|nice|fantastic|best|amaze|amazing|awesome|adorable|perfectly)\b','good',case=False)


C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_52072/1014543160.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_en['Review']=data_en['Review'].str.replace(r"\b(?:cancel|cancelled|canceled)\b",'cancel',case=False)
C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_52072/1014543160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_en['Review']=data_en['Review'].str.replace(r"\b(?:cancel|cancelled|canceled)\b",'cancel',case=False)
C:\Users\YUFENG~1\AppData\Local\Temp/ipykernel_52072/1014543160.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_en['Review']=data_en['Review'].str.replace(r"\b(?:location|position)\b",'location',case=Fal

## Output Clean Data

In [60]:
clean_data=data_en.loc[(data_en['App']=='Bumble') |(data_en['App']=='Hinge') ,:]

In [61]:
clean_data.to_csv("bumble_hinge_review.csv",index=None)